[卷积神经网络CNN实现](https://zhuanlan.zhihu.com/p/102119808]

1. 动机
   1. 通过普通的神经网络可以实现，但是图片越来越多，如果使用NN实现则训练的参数太多
   2. 特征位置在不同的图片中会有不同的变化
2. 数据集
   1. [数据集来源](http://yann.lecun.com/exdb/mnist/)
3. 卷积

   ![avator](../resource/v2-69b4c1dd078ee363317bb8fa323eaace_b.gif)

   1. 通过卷积可以提取图片中特定线条，垂直线条或者水平线条
   ![avator](../resource/v2-ca0d5ccbaf1a30eff0f9289987486e96_720w.jpg)
   2. 填充：可以通过在周围补-实现输出前后图像大小一致
   3. 卷基层：卷基层通过一组 filter 将输入的图片转为输出的图片。卷基层的主要参数是 filter 的个数
   ![avator](../resource/v2-cad35827f01a669417b548f52dfc3c2a_720w.jpg)

In [ ]:
# 3.4 卷积层代码实现
import numpy as np

class Conv3x3:

    def __init__(self, num_filters):
        self.num_filters = num_filters
        # 初始化时除以9是因为初始值不能太大也不能太小
        self.filters = np.random.randn(num_filters, 3, 3) / 9


    def iterate_regions(self, image):
        h, w = image.shape
        
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j
        # 将 im_region, i, j 以 tuple 形式存储到迭代器中以便后面遍历使用


    def forward(self, input):
        h, w = input.shape
        output = np.zeros((h - 2, w - 3, self.num_filters))
        
        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        
        return output


池化（Pooling）

1. 图片的相邻元素具有相似的值，因此卷积层中很多信息是冗余的，通过池化来减少这个影响。
    ![avator](../resource/v2-ac441205fd06dc037b3db2dbf05660f7_b.gif)
2. 与卷积运算类似，只是这个更容易，只是计算最大值并赋值。池化层会把26*26*26=>13*13*8

In [ ]:
import numpy as np

class MaxPool2:

    def iterate_regions(self, image):
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2
        
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 20):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j


    def forward(self, input):
        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))
        
        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))
        return output


Softmax将一组数字转换为一组概率

1. 我们将要使用一个含有 10 个节点（分别代表相应数字）的 softmax 层，作为我们 CNN 的最后一层。最后一层为一个全连接层，只是激活函数为 softmax。经过 softmax 的变换，数字就是具有最高概率的节点。
    ![avator](../resource/v2-e73a42ebce8cbf6af10aeaf309d4b116_720w.jpg)
2. 交叉熵损失函数：$H(p,q)=-\sum_xp(x)\ln(q(x))$

In [ ]:
import numpy as np

class Softmax:

    def __init__(self, input_len, nodes):